## Obtener el error de picota, leyendo el pdf de resulatdos

In [26]:
import fitz  # PyMuPDF

doc = fitz.open("../../data/report.pdf")

for page_num, page in enumerate(doc, start=1):
    text = page.get_text()
    print(f"Página {page_num}:")
    print(text)
    print("-" * 80)


Página 1:
Inferential DataSheet
picota
Model Identifier: 6c0d63e8-0ca5-45ac-bc2f-316db77f921a
https://quassar.io/commits/6c0d63e8-0ca5-45ac-bc2f-316db77f921a
Title: main_48_0_1789027358   Release Date: 2025-06-30
Digital Subject: hogar07
Estimation: 
gastoMonetario:productosAlimenticios11
Inference Error: ±2,53 %
Top Contributing Inputs
Input Variable
Weight (%)
ingresosNetos
16,00%
tasaAhorro
15,00%
comidasTotales
15,00%
ipc
9,00%
Estimation: 
gastoMonetario:bebidasNoAlcoholicas12
Inference Error: ±2,11 %
Top Contributing Inputs
Input Variable
Weight (%)
tasaAhorro
16,00%
ingresosNetos
14,00%
ipc
9,00%
comidasTotales
9,00%
Estimation: 
gastoMonetario:bebidasAlcoholicas21
Inference Error: ±,32 %
Top Contributing Inputs
Estimation: gastoMonetario:tabaco22
Inference Error: ±2,57 %
Top Contributing Inputs
Input Variable
Weight (%)
tasaAhorro
14,00%

--------------------------------------------------------------------------------
Página 2:
Input Variable
Weight (%)
tasaAhorro
16,00%
ingres

In [32]:
import fitz
import re
import pandas as pd

# Ruta a tu PDF
ruta_pdf = "../../data/report.pdf"

# Abrir documento
doc = fitz.open(ruta_pdf)

data = []
subject_actual = None

# Recorrer páginas
for page_num, page in enumerate(doc, start=1):
    text = page.get_text()

    # Buscar Digital Subject en la página
    m_subject = re.search(r"Digital Subject:\s*(\w+)", text)
    if m_subject:
        subject_actual = m_subject.group(1)

    # Buscar bloques de Estimation
    bloques = re.findall(
        r"Estimation:\s*(gastoMonetario:[^\n]*)\nInference Error:\s*±([\d,\.]+)\s*%",
        text
    )

    for var, err in bloques:
        # Quitar prefijo "gastoMonetario:"
        var_clean = var.replace("gastoMonetario:", "")
        # Normalizar decimal (puede venir coma o punto)
        err_val = float(err.replace(",", "."))
        data.append({
            "subject": subject_actual,
            "gastoMonetario": var_clean,
            "inference_error_pct": err_val
        })

# Pasar a DataFrame
df_errors = pd.DataFrame(data)

print(df_errors)


     subject                         gastoMonetario  inference_error_pct
0    hogar07                productosAlimenticios11                 2.53
1    hogar07                 bebidasNoAlcoholicas12                 2.11
2    hogar07                   bebidasAlcoholicas21                 0.32
3    hogar07                               tabaco22                 2.57
4    hogar07                    articulosDeVestir31                 1.31
5    hogar07                              calzado32                 1.69
6    hogar07         alquileresRealesDeLaVivienda41                 1.20
7    hogar07            mantenimientoDeLaVivienda43                 0.59
8    hogar07                     suministroDeAgua44                 1.46
9    hogar07     electricidadGasOtrosCombustibles45                 1.68
10   hogar07                mueblesRevestimientos51                 1.04
11   hogar07                  textilesParaElHogar52                 0.96
12   hogar07             grandesElectrodomesticos53

In [33]:
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
df_errors

,subject,gastoMonetario,inference_error_pct
0,hogar07,productosAlimenticios11,2.53
1,hogar07,bebidasNoAlcoholicas12,2.11
2,hogar07,bebidasAlcoholicas21,0.32
3,hogar07,tabaco22,2.57
4,hogar07,articulosDeVestir31,1.31
5,hogar07,calzado32,1.69
6,hogar07,alquileresRealesDeLaVivienda41,1.20
7,hogar07,mantenimientoDeLaVivienda43,0.59
8,hogar07,suministroDeAgua44,1.46
9,hogar07,electricidadGasOtrosCombustibles45,1.68


In [34]:
# Guardar el DataFrame final en un archivo TSV
df_errors.to_csv("../../datamarts/inference_errors.tsv", sep="\t", index=False)